# Quickstart for Carsus

In this guide you will learn how to create atomic data files suitable for [TARDIS](https://github.com/tardis-sn/tardis).

<div class="alert alert-info">

**Note:**

Get familiar with the [Notation in Carsus](development/notation.rst) and learn how to correctly select ions.
    
</div>

## Atomic Weights and Ionization Energies (NIST)

Download atomic weights and ionization energies from the National Institute of Standards and Technology (NIST).

In [ ]:
from carsus.io.nist import NISTWeightsComp, NISTIonizationEnergies

In [ ]:
atomic_weights = NISTWeightsComp()
ionization_energies = NISTIonizationEnergies('H-Zn')

## Levels, Lines, Collisions & Cross-sections

Carsus supports three sources of energy levels and spectral lines: **GFALL**, **CHIANTI** and **CMFGEN**.

### GFALL

The Robert Kurucz's Atomic Linelist (GFALL) reader is the main source of **levels and lines**.

<div class="alert alert-warning">

**Warning:**
    
Creating a `GFALLReader` instance is **required**.

</div>

In [ ]:
!wget -qO /tmp/gfall.dat https://media.githubusercontent.com/media/tardis-sn/carsus-db/master/gfall/gfall_latest.dat

In [ ]:
from carsus.io.kurucz import GFALLReader

gfall_reader = GFALLReader('H-Zn',
                           '/tmp/gfall.dat')

### CHIANTI

The Chianti Atomic Database reader provides levels and lines but also **collision strengths**.

<div class="alert alert-info">

**Note:**

Creating a `ChiantiReader` instance is **optional**. 

</div>

In [ ]:
from carsus.io.chianti_ import ChiantiReader

chianti_reader = ChiantiReader('H-He', 
                               collisions=True, 
                               priority=20)

By default `priority` parameter is set to `10`. Increase this value if you want to keep CHIANTI levels and lines over GFALL.

### CMFGEN

The atomic database of [CMFGEN](http://kookaburra.phyast.pitt.edu/hillier/web/CMFGEN.htm) is a source of levels, lines and (optionally) **ionization energies**, **photoionization cross-sections** and **collisions**.

<div class="alert alert-info">
    
**Note:**

Creating a `CMFGENReader` instance is **optional**. 

</div>

<div class="alert alert-warning">

**Warning:**
    
Cross-sections require data from `H 0`, use this the reader with enough `priority` to select levels from this ion.

</div>

In [ ]:
from carsus.io.cmfgen import CMFGENReader

cmfgen_reader = CMFGENReader.from_config('Si 0-1', 
                                         '/tmp/atomic', 
                                         priority=30,
                                         ionization_energies=True,
                                         cross_sections=True,
                                         collisions=False,
                                         temperature_grid=None,
                                         drop_mismatched_labels=True)

## Zeta Data

Long & Knigge's ground state recombinations fractions ($\zeta$).

In [ ]:
from carsus.io.zeta import KnoxLongZeta

zeta_data = KnoxLongZeta()

## Create an Atomic Data File

Finally, create a `TARDISAtomData` object and dump the data with the `to_hdf` method.

In [ ]:
from carsus.io.output import TARDISAtomData

atom_data = TARDISAtomData(atomic_weights,
                           ionization_energies,
                           gfall_reader,
                           zeta_data,
                           chianti_reader,
                           cmfgen_reader)

In [ ]:
atom_data.to_hdf('kurucz_cd23_chianti_He_cmfgen_H_Si_I-II.h5')

You are done! Now you can use your file to run TARDIS simulations.

### Metadata

Carsus stores metadata inside the HDF5 files to ensure reproducibility. This metadata includes a checksum for each stored table, version number or checksum of selected datasets, and versions of relevant packages. 

In [ ]:
import pandas as pd

In [ ]:
store = pd.HDFStore('kurucz_cd23_chianti_He_cmfgen_H_Si_I-II.h5', key='metadata')

In [ ]:
store["metadata"]

In [ ]:
store.root._v_attrs